CPD builder

In [70]:
import pandas as pd
from pgmpy.factors.discrete import TabularCPD

class BayesianCPTBuilder:
    def __init__(self, filepath):
        self.df = pd.read_csv(filepath)
        self.df = self.df.rename(columns={
            'gender': 'Gender',
            'feature': 'Feature',
            'median_income': 'IncomeLevel',
            'PROPORTION': 'Proportion'
        })
        self.genders = sorted(self.df['Gender'].unique())
        self.feature_groups = sorted(self.df['Feature'].unique())
        self.income_levels = sorted(self.df['IncomeLevel'].unique())
        self.income_levels = ['<10k', '10k-20k', '20k-30k', '30k-50k', '50k-70k', '70k-90k', '90k-110k', '110k-130k', '130-150k', '>150k']

    def build_feature_given_gender_cpd(self, feature_name):
        cpt_matrix = []
        for feature in self.feature_groups:
            row = []
            for gender in self.genders:
                prob = self.df[
                    (self.df['Gender'] == gender) & (self.df['Feature'] == feature)
                ]['Proportion'].values
                row.append(prob[0] if len(prob) > 0 else 0.0)
            cpt_matrix.append(row)

        return TabularCPD(
            variable=feature_name,
            variable_card=len(self.feature_groups),
            values=cpt_matrix,
            evidence=['Gender'],
            evidence_card=[len(self.genders)],
            state_names={
                feature_name: self.feature_groups,
                'Gender': self.genders
            }
        )
        
    def build_income_given_feature_gender_cpd(self, feature_name):
        cpt_matrix = []
        for income in self.income_levels:
            row = []
            for gender in self.genders:
                for feature in self.feature_groups:
                    prob = self.df[
                        (self.df['Gender'] == gender) &
                        (self.df['Feature'] == feature) &
                        (self.df['IncomeLevel'] == income)
                    ]['Proportion'].sum()
                    row.append(prob)
            cpt_matrix.append(row)

        import numpy as np
        cpt_array = np.array(cpt_matrix)
        col_sums = cpt_array.sum(axis=0)
        normalized_matrix = (cpt_array / col_sums).tolist()

        return TabularCPD(
            variable='IncomeLevel',
            variable_card=len(self.income_levels),
            values=normalized_matrix,
            evidence=[feature_name, 'Gender'],
            evidence_card=[len(self.feature_groups), len(self.genders)],
            state_names={
                'IncomeLevel': self.income_levels,
                feature_name: self.feature_groups,
                'Gender': self.genders
            }
        )




In [71]:
##CPT可视化热力图 TODO

# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt

# # 读取整理后的 CPT 数据
# df = pd.read_csv("Age_group_gender_transformed.csv")

# # 重命名列方便处理
# df = df.rename(columns={'feature': 'Feature', 'gender': 'Gender', 'PROPORTION': 'Probability'})

# # 创建透视表：行是年龄组，列是性别，值是概率
# pivot_df = df.pivot(index='Feature', columns='Gender', values='Probability')

# # 设置图形大小
# plt.figure(figsize=(8, 10))

# # 绘制热力图
# sns.heatmap(
#     pivot_df,
#     annot=True, fmt=".2f", cmap="YlGnBu",
#     cbar_kws={'label': 'P(Feature | Gender)'}
# )

# # 设置标题和轴标签
# plt.title("Conditional Probability: P(Feature | Gender)", fontsize=14)
# plt.xlabel("Gender")
# plt.ylabel("Age Group")

# # 显示图形
# plt.tight_layout()
# plt.show()


In [72]:
import numpy as np
from pgmpy.factors.discrete import TabularCPD

def fix_cpd_normalization(cpd: TabularCPD) -> TabularCPD:
    arr = np.array(cpd.values)
    # arr: shape = (variable_card, n_cols)
    for col in range(arr.shape[1]):
        s = arr[:, col].sum()
        if np.isclose(s, 0):
            arr[:, col] = 1.0 / arr.shape[0]
        else:
            arr[:, col] = arr[:, col] / s
    return TabularCPD(
        variable=cpd.variable,
        variable_card=cpd.variable_card,
        values=arr.tolist(),
        evidence=cpd.variables[1:],
        evidence_card=cpd.cardinality[1:],
        state_names=cpd.state_names
    )


constructing cpd and corresponding BN

In [73]:
# 构建BN框架
from pgmpy.models import DiscreteBayesianNetwork

builder_age = BayesianCPTBuilder("../../data/randomized marginal tables/age_gender_randomized.csv")
gender_proportions = builder_age.df.groupby('Gender')['Proportion'].sum()
cpd_age_gender = builder_age.build_feature_given_gender_cpd('AgeGroup')
cpd_age_gender = fix_cpd_normalization(cpd_age_gender)
cpd_age_gender.normalize()

builder_industry = BayesianCPTBuilder("../../data/randomized marginal tables/industry_gender_randomized.csv")
cpd_industry_gender = builder_industry.build_feature_given_gender_cpd('Industry')
cpd_industry_gender = fix_cpd_normalization(cpd_industry_gender)
cpd_industry_gender.normalize()

builder_corp_size = BayesianCPTBuilder("../../data/randomized marginal tables/corpSize_gender_randomized.csv")
cpd_corp_size_gender = builder_corp_size.build_feature_given_gender_cpd('CorpSize')
cpd_corp_size_gender = fix_cpd_normalization(cpd_corp_size_gender)
cpd_corp_size_gender.normalize()

builder_employmentType = BayesianCPTBuilder("../../data/randomized marginal tables/employmentType_gender_randomized.csv")
cpd_employment_type_gender = builder_employmentType.build_feature_given_gender_cpd('EmploymentType')
cpd_employment_type_gender = fix_cpd_normalization(cpd_employment_type_gender)
cpd_employment_type_gender.normalize()

builder_legal_org = BayesianCPTBuilder("../../data/randomized marginal tables/legal_org_gender_randomized.csv")
cpd_legal_org_gender = builder_legal_org.build_feature_given_gender_cpd('LegalOrganization')
cpd_legal_org_gender = fix_cpd_normalization(cpd_legal_org_gender)
cpd_legal_org_gender.normalize()

builder_sector = BayesianCPTBuilder("../../data/randomized marginal tables/sector_gender_randomized.csv")
cpd_sector_gender = builder_sector.build_feature_given_gender_cpd('Sector')
cpd_sector_gender = fix_cpd_normalization(cpd_sector_gender)
cpd_sector_gender.normalize()

cpd_gender = TabularCPD(
    variable='Gender',
    variable_card=2,
    values=[[gender_proportions['FEMALES']], [gender_proportions['MALES']]],
    state_names={'Gender': ['FEMALES', 'MALES']}
)
cpd_gender.normalize()

# 定义结构：Gender → Feature
model = DiscreteBayesianNetwork()
model.add_edges_from([
    ('Gender', 'AgeGroup'),
    ('Gender', 'Industry'),
    ('Gender', 'CorpSize'),
    ('Gender', 'EmploymentType'),
    ('Gender', 'LegalOrganization'),
    ('Gender', 'Sector'),
    ('AgeGroup', 'IncomeLevel'),
    ('Industry', 'IncomeLevel'),
    ('CorpSize', 'IncomeLevel'),
    ('EmploymentType', 'IncomeLevel'),
    ('LegalOrganization', 'IncomeLevel'),
    ('Sector', 'IncomeLevel')
])

model.add_cpds(cpd_gender) # 加入先验gender概率
model.add_cpds(cpd_age_gender, cpd_industry_gender)
model.add_cpds(cpd_corp_size_gender, cpd_employment_type_gender, cpd_legal_org_gender, cpd_sector_gender)


# 联合概率构建尝试

In [74]:
import pandas as pd
import itertools

# 1. 先分别准备每个特征的概率表
def get_prob_table(builder, feature_col):
    prob_df = builder.df[['Feature', 'Gender', 'IncomeLevel', 'Proportion']].copy()
    prob_df = prob_df.rename(columns={
        'Feature': feature_col,
        'Proportion': f'Prob_{feature_col}'
    })
    return prob_df

prob_age = get_prob_table(builder_age, 'AgeGroup')
prob_industry = get_prob_table(builder_industry, 'Industry')
prob_corp_size = get_prob_table(builder_corp_size, 'CorpSize')
prob_employment_type = get_prob_table(builder_employmentType, 'EmploymentType')
prob_legal_org = get_prob_table(builder_legal_org, 'LegalOrganization')
prob_sector = get_prob_table(builder_sector, 'Sector')

In [75]:
# 2. 依次 merge 到 df_joint7
age_groups = sorted(builder_age.df['Feature'].unique())
industry_groups = sorted(builder_industry.df['Feature'].unique())
corp_size_groups = sorted(builder_corp_size.df['Feature'].unique())
employment_types = sorted(builder_employmentType.df['Feature'].unique())
legal_orgs = sorted(builder_legal_org.df['Feature'].unique())
sectors = sorted(builder_sector.df['Feature'].unique())
genders = sorted(builder_age.df['Gender'].unique())
income_levels = sorted(builder_age.df['IncomeLevel'].unique())

combinations = list(itertools.product(
    age_groups, industry_groups, corp_size_groups,
    employment_types, legal_orgs, sectors, genders, income_levels
))
df_joint7 = pd.DataFrame(combinations, columns=[
    'AgeGroup', 'Industry', 'CorpSize', 'EmploymentType',
    'LegalOrganization', 'Sector', 'Gender', 'IncomeLevel'
])

df = df_joint7
df = df.merge(prob_age, on=['AgeGroup', 'Gender', 'IncomeLevel'], how='left')
df = df.merge(prob_industry, on=['Industry', 'Gender', 'IncomeLevel'], how='left')
df = df.merge(prob_corp_size, on=['CorpSize', 'Gender', 'IncomeLevel'], how='left')
df = df.merge(prob_employment_type, on=['EmploymentType', 'Gender', 'IncomeLevel'], how='left')
df = df.merge(prob_legal_org, on=['LegalOrganization', 'Gender', 'IncomeLevel'], how='left')
df = df.merge(prob_sector, on=['Sector', 'Gender', 'IncomeLevel'], how='left')

In [76]:
# 3. 直接相乘得到联合概率
df['Proportion'] = (
    df['Prob_AgeGroup'].fillna(0) *
    df['Prob_Industry'].fillna(0) *
    df['Prob_CorpSize'].fillna(0) *
    df['Prob_EmploymentType'].fillna(0) *
    df['Prob_LegalOrganization'].fillna(0) *
    df['Prob_Sector'].fillna(0)
)

# 4. 归一化
group_cols = ['AgeGroup', 'Industry', 'CorpSize', 'EmploymentType', 'LegalOrganization', 'Sector', 'Gender']
df['Proportion'] = df.groupby(group_cols)['Proportion'].transform(lambda x: x / x.sum())

df_joint7 = df

In [77]:
def build_income_given_all_features_cpd_fast(
    df_joint7, income_levels, age_groups, industry_groups, corp_size_groups,
    employment_types, legal_orgs, sectors
):
    all_index = pd.MultiIndex.from_product(
        [income_levels, age_groups, industry_groups, corp_size_groups, employment_types, legal_orgs, sectors],
        names=['IncomeLevel', 'AgeGroup', 'Industry', 'CorpSize', 'EmploymentType', 'LegalOrganization', 'Sector']
    )
    pivot = df_joint7.pivot_table(
        index=['IncomeLevel', 'AgeGroup', 'Industry', 'CorpSize', 'EmploymentType', 'LegalOrganization', 'Sector'],
        values='Proportion',
        fill_value=0
    ).reindex(all_index, fill_value=0)

    shape = (
        len(income_levels),
        len(age_groups),
        len(industry_groups),
        len(corp_size_groups),
        len(employment_types),
        len(legal_orgs),
        len(sectors)
    )
    arr = pivot.values.reshape(shape)
    cpt_matrix = arr.reshape(len(income_levels), -1)

    # 列归一化，并强制每列和为1
    import numpy as np
    col_sums = cpt_matrix.sum(axis=0)
    normalized_matrix = []
    for col in range(cpt_matrix.shape[1]):
        col_sum = col_sums[col]
        if np.isclose(col_sum, 0):
            # 如果全为0，填均匀分布
            normalized_matrix.append([1.0 / len(income_levels)] * len(income_levels))
        else:
            normalized_matrix.append((cpt_matrix[:, col] / col_sum).tolist())
    # 转置回 income_levels 行，其余组合为列
    normalized_matrix = np.array(normalized_matrix).T.tolist()

    return TabularCPD(
        variable='IncomeLevel',
        variable_card=len(income_levels),
        values=normalized_matrix,
        evidence=['AgeGroup', 'Industry', 'CorpSize', 'EmploymentType', 'LegalOrganization', 'Sector'],
        evidence_card=[
            len(age_groups), len(industry_groups), len(corp_size_groups),
            len(employment_types), len(legal_orgs), len(sectors)
        ],
        state_names={
            'IncomeLevel': income_levels,
            'AgeGroup': age_groups,
            'Industry': industry_groups,
            'CorpSize': corp_size_groups,
            'EmploymentType': employment_types,
            'LegalOrganization': legal_orgs,
            'Sector': sectors
        }
    )
    
cpd_income_all = build_income_given_all_features_cpd_fast(
    df_joint7, income_levels, age_groups, industry_groups, corp_size_groups,
    employment_types, legal_orgs, sectors
)

model.add_cpds(cpd_income_all) # 加入联合概率cpd -> income

In [78]:
# 检查模型是否有效（所有节点都有 CPT，结构无环）
assert model.check_model()

In [57]:
from pgmpy.sampling import BayesianModelSampling

# 假设你的模型变量名为 model
sampling_inference = BayesianModelSampling(model)

# 采样100000组数据
samples2 = sampling_inference.forward_sample(size=100000)
print(samples2.head())
samples2.to_csv("test_samples_100000.csv", index=False)


  0%|          | 0/8 [00:00<?, ?it/s]

    Gender        AgeGroup                             Industry  \
0    MALES  60 to 64 years  Administrative and support services   
1  FEMALES  55 to 59 years               Education and training   
2  FEMALES  21 to 24 years                         Construction   
3  FEMALES  30 to 34 years    Health care and social assistance   
4    MALES  15 to 17 years     Public administration and safety   

                 CorpSize                          EmploymentType  \
0   200 or more employees                  Public sector entities   
1        20–199 employees    Incorporated Private sector entities   
2   200 or more employees                  Public sector entities   
3  Fewer than 5 employees  Unincorporated Private sector entities   
4          5–19 employees    Incorporated Private sector entities   

                            LegalOrganization                      Sector  \
0                                  Households  Non-financial corporations   
1                           

In [58]:
# 检查samples2中每个特征的概率分布
features = ['Gender', 'AgeGroup', 'Industry', 'CorpSize', 'EmploymentType', 'LegalOrganization', 'Sector', 'IncomeLevel']

for col in features:
    print(f"\n{col} 分布：")
    print(samples2[col].value_counts(normalize=True))


Gender 分布：
MALES      0.51305
FEMALES    0.48695
Name: Gender, dtype: float64

AgeGroup 分布：
25 to 29 years        0.13297
30 to 34 years        0.12392
35 to 39 years        0.10890
21 to 24 years        0.09831
40 to 44 years        0.09570
45 to 49 years        0.09358
50 to 54 years        0.08543
55 to 59 years        0.07323
18 to 20 years        0.05876
60 to 64 years        0.05530
65 to 69 years        0.02919
15 to 17 years        0.02240
70 to 74 years        0.01215
75 to 79 years        0.00473
80 to 84 years        0.00255
85 years and over     0.00150
14 years and under    0.00138
Name: AgeGroup, dtype: float64

Industry 分布：
Health care and social assistance                  0.12957
Administrative and support services                0.10028
Retail trade                                       0.09627
Accommodation and food services                    0.08453
Professional, scientific and technical services    0.08443
Education and training                             0.0816